In [2]:
#SparkSession의 연결을 끊기 위하여
quit()

: 

# Spark 데이터 파이프라인 - yarn

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

## SparkSession

- spark의 기능의 시작점
- 데이터 프레임 기능을 사용하기 위함

In [4]:
%%time
yarn = SparkSession\
        .builder\
        .master('yarn')\
        .appName('twitter_yarn')\
        .config('spark.submit.deployMode', 'client')\
        .config('spark.executor.instances', 4)\
        .config('spark.executor.memory', '2g')\
        .config('spark.driver.memory', '2g')\
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
        .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bigdata/.ivy2/cache
The jars for the packages stored in: /home/bigdata/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-31e3b474-dc24-482e-8a47-a6531e5c5985;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 258ms :: artifacts dl 9ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifact

23/03/27 17:49:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 17:49:51 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/03/27 17:50:08 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.12-3.0.1.jar added multiple times to distributed cache.
23/03/27 17:50:08 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb_mongodb-driver-sync-4.0.5.jar added multiple times to distributed cache.
23/03/27 17:50:08 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb_bson-4.0.5.jar added multiple times to distributed cache.
23/03/27 17:50:08 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb_mongodb-driver-core-4.0.5.jar added multiple times to distributed cache.
CPU times: user 15.4 ms, sys: 75 ms, total: 90.4 ms
Wall time: 37.2 s


In [5]:
yarn.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '4'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'),
 ('spark.driver.appUIAddress', 'http://master:4040'),
 ('spark.driver.port', '34

## 데이터 파이프라인

In [6]:
%%time
df = yarn.read.format('mongo')\
               .option('uri', 'mongodb://192.168.56.101:27017/twitter.sample')\
               .load()

df.createOrReplaceTempView('tweets')

CPU times: user 0 ns, sys: 16.3 ms, total: 16.3 ms
Wall time: 17.8 s


In [7]:
%%time

type(df)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.01 µs


pyspark.sql.dataframe.DataFrame

mongoDB로부터 저장되어있는 데이터를 불러오고 데이터프레임의 형태로 저장합니다.

`createOrReplaceTempView`함수로 `sql`을 사용하여 핸들링할 수 있도록 `view`에 `tweets`라는 이름으로 등록합니다.

### 전체 데이터 수 확인

#### sql을 사용하여 조회

In [8]:
%%time
query ='''
SELECT count(*)
FROM tweets
'''
yarn.sql(query).show()

+--------+
|count(1)|
+--------+
| 7739109|
+--------+

CPU times: user 53.1 ms, sys: 101 ms, total: 154 ms
Wall time: 1min 4s


#### Dataframe 함수를 사용하여 조회

In [9]:
#d
%%time
df.count()

CPU times: user 74.5 ms, sys: 114 ms, total: 188 ms
Wall time: 37.7 s


7739109

전체 데이터의 수는 약 770만개입니다.

### 상위 5개 데이터 조회

#### Sql 사용하여 조회

In [10]:
%%time
# 데이터 확인

query = """
  SELECT *
  FROM tweets
"""
yarn.sql(query).show(5)

+--------------------+--------------------+--------------------+
|                 _id|          _timestamp|                data|
+--------------------+--------------------+--------------------+
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

CPU times: user 645 µs, sys: 9.03 ms, total: 9.68 ms
Wall time: 2.95 s


#### Dataframe 함수를 사용하여 조회

In [25]:
%%time
df.show(5)

+--------------------+--------------------+--------------------+
|                 _id|          _timestamp|                data|
+--------------------+--------------------+--------------------+
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

CPU times: user 2.02 ms, sys: 2.16 ms, total: 4.18 ms
Wall time: 298 ms


##### df.head()

In [16]:
df.head(5)

[Row(_id=Row(oid='64102551db4069eff534f0c2'), _timestamp='2023-03-14T16:42:09.451647+09:00', data=Row(created_at='2023-03-14T07:42:03.000Z', edit_history_tweet_ids=['1635546820548993024'], id='1635546820548993024', lang='fr', text='RT @Mweilc: Après @AmiensMetropole fin de parcours des tableaux de la Chambre du Roi enfin visibles à hauteur d’homme  @CVersailles avec en…', withheld=None)),
 Row(_id=Row(oid='64102551db4069eff534f0c3'), _timestamp='2023-03-14T16:42:09.474800+09:00', data=Row(created_at='2023-03-14T07:42:03.000Z', edit_history_tweet_ids=['1635546820523728897'], id='1635546820523728897', lang='en', text='@erdincciftci Good morning forever Erdin 🙏🔥', withheld=None)),
 Row(_id=Row(oid='64102551db4069eff534f0c4'), _timestamp='2023-03-14T16:42:09.476270+09:00', data=Row(created_at='2023-03-14T07:42:03.000Z', edit_history_tweet_ids=['1635546820540788736'], id='1635546820540788736', lang='en', text='@CheapremeBrand Am i right https://t.co/34FUg5Ed7e', withheld=None)),
 Row(_id=Ro

In [15]:
type(df.head(5)[0])

pyspark.sql.types.Row

`df.head(5)`를 통하여 조회할 수 있지만 `df.head(5)`는 로우 오브젝트를 반환합니다.

### 언어별 count 확인

#### Sql 사용하여 조회

In [17]:
%%time
#언어별 count 확인

query = '''
  SELECT data.lang as lang, count(*) count
  FROM tweets
  WHERE data.lang is not NULL
  GROUP BY lang
  ORDER BY count DESC;
'''

yarn.sql(query).show(5)

+----+-------+
|lang|  count|
+----+-------+
|  en|2197064|
|  ja|1708222|
|  th| 484936|
|  es| 467246|
|  ko| 316200|
+----+-------+
only showing top 5 rows

CPU times: user 75.3 ms, sys: 125 ms, total: 200 ms
Wall time: 39.5 s


#### Dataframe 함수를 사용하여 조회

In [20]:
%%time
df.groupby('data.lang').count().show(5)

+----+-------+
|lang|  count|
+----+-------+
| art|  15009|
|  en|2197064|
|  vi|   8691|
|  ne|   2113|
|  ps|   1232|
+----+-------+
only showing top 5 rows

CPU times: user 102 ms, sys: 60.5 ms, total: 162 ms
Wall time: 34.9 s


### 영어로 작성된 tweet만 추출

#### Sql 사용

In [21]:
%%time
#영어로 작성된 tweet만 추출

query = """
  SELECT
    DATE_FORMAT(_timestamp, "yyyy-MM-dd HH:mm:ss") as time,
    data.text as text
  FROM tweets
  WHERE data.lang = 'en';
"""

en_tweets_sql = yarn.sql(query)
en_tweets_sql.show(5)

+-------------------+--------------------+
|               time|                text|
+-------------------+--------------------+
|2023-03-14 16:42:09|@erdincciftci Goo...|
|2023-03-14 16:42:09|@CheapremeBrand A...|
|2023-03-14 16:42:09|RT @Joon_record: ...|
|2023-03-14 16:42:09|RT @TheRavenCrypt...|
|2023-03-14 16:42:09|@gareth_lane He i...|
+-------------------+--------------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 5.94 ms, total: 5.94 ms
Wall time: 812 ms


#### DataFrame 함수 사용

In [91]:
%%time
from pyspark.sql.functions import date_format
en_tweets_sdf = df[['_timestamp', 'data.text']].withColumnRenamed('_timestamp','time')
en_tweets_sdf = en_tweets_sdf.select(date_format('time', 'yyyy-MM-dd HH:mm:ss').alias('time'), en_tweets_sdf.text)
en_tweets_sdf.show(5)

+-------------------+--------------------+
|               time|                text|
+-------------------+--------------------+
|2023-03-14 16:42:09|RT @Mweilc: Après...|
|2023-03-14 16:42:09|@erdincciftci Goo...|
|2023-03-14 16:42:09|@CheapremeBrand A...|
|2023-03-14 16:42:09|@homerhsimpson Ka...|
|2023-03-14 16:42:09|RT @Joon_record: ...|
+-------------------+--------------------+
only showing top 5 rows

CPU times: user 2.55 ms, sys: 11.7 ms, total: 14.2 ms
Wall time: 2.96 s


### 샘플 추출

In [92]:
%%time
#샘플하나 추출
en_tweets_sql.take(1)

CPU times: user 0 ns, sys: 9.91 ms, total: 9.91 ms
Wall time: 2.75 s


[Row(time='2023-03-14 16:42:09', text='@erdincciftci Good morning forever Erdin 🙏🔥')]

In [93]:
%%time
en_tweets_sdf.take(1)

CPU times: user 0 ns, sys: 11.6 ms, total: 11.6 ms
Wall time: 2.52 s


[Row(time='2023-03-14 16:42:09', text='RT @Mweilc: Après @AmiensMetropole fin de parcours des tableaux de la Chambre du Roi enfin visibles à hauteur d’homme  @CVersailles avec en…')]

### tweet의 text를 단어별로 split하는 함수
- spark의 함수를 사용하여 진행

In [94]:
%%time
#tweet의 text를 단어별로 split하는 함수
def text_split(row):
  for word in row.text.split():
    yield Row(time=row.time, word=word)
  

CPU times: user 8 µs, sys: 4 µs, total: 12 µs
Wall time: 13.6 µs


#### en_tweets_sql 

In [95]:
%%time
#단어별로 split한뒤에 5개 show
en_tweets_sql.rdd.flatMap(text_split).take(5)

CPU times: user 5.99 ms, sys: 13.5 ms, total: 19.5 ms
Wall time: 2.07 s


[Row(time='2023-03-14 16:42:09', word='@erdincciftci'),
 Row(time='2023-03-14 16:42:09', word='Good'),
 Row(time='2023-03-14 16:42:09', word='morning'),
 Row(time='2023-03-14 16:42:09', word='forever'),
 Row(time='2023-03-14 16:42:09', word='Erdin')]

In [98]:
%%time
#단어별로 split한뒤에 데이터프레임으로 변경후 5개 show
en_tweets_sql.rdd.flatMap(text_split).toDF().show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-14 16:42:09|@erdincciftci|
|2023-03-14 16:42:09|         Good|
|2023-03-14 16:42:09|      morning|
|2023-03-14 16:42:09|      forever|
|2023-03-14 16:42:09|        Erdin|
+-------------------+-------------+
only showing top 5 rows

CPU times: user 100 ms, sys: 218 ms, total: 318 ms
Wall time: 5.16 s


#### en_tweets_sdf

In [96]:
%%time
en_tweets_sdf.rdd.flatMap(text_split).take(5)

CPU times: user 4.8 ms, sys: 16.1 ms, total: 20.9 ms
Wall time: 2.51 s


[Row(time='2023-03-14 16:42:09', word='RT'),
 Row(time='2023-03-14 16:42:09', word='@Mweilc:'),
 Row(time='2023-03-14 16:42:09', word='Après'),
 Row(time='2023-03-14 16:42:09', word='@AmiensMetropole'),
 Row(time='2023-03-14 16:42:09', word='fin')]

In [99]:
%%time
#단어별로 split한뒤에 데이터프레임으로 변경후 5개 show
en_tweets_sdf.rdd.flatMap(text_split).toDF().show(5)

+-------------------+----------------+
|               time|            word|
+-------------------+----------------+
|2023-03-14 16:42:09|              RT|
|2023-03-14 16:42:09|        @Mweilc:|
|2023-03-14 16:42:09|           Après|
|2023-03-14 16:42:09|@AmiensMetropole|
|2023-03-14 16:42:09|             fin|
+-------------------+----------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 29.2 ms, total: 29.2 ms
Wall time: 3.89 s


### 단어별로 분리하고 데이터프레임으로 추출

#### Sql 사용

In [101]:
%%time
#단어별로 분리하고 데이터프레임으로 추출
words_sql = en_tweets_sql.rdd.flatMap(text_split).toDF()
words_sql.createOrReplaceTempView('words_sql')


CPU times: user 8.81 ms, sys: 20.5 ms, total: 29.4 ms
Wall time: 2.16 s


In [105]:
%%time
#words의 상위 5개 show
query = '''
    SELECT *
    FROM words_sql
'''

yarn.sql(query).show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-14 16:42:09|@erdincciftci|
|2023-03-14 16:42:09|         Good|
|2023-03-14 16:42:09|      morning|
|2023-03-14 16:42:09|      forever|
|2023-03-14 16:42:09|        Erdin|
+-------------------+-------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 10.5 ms, total: 10.5 ms
Wall time: 2.1 s


In [106]:
%%time
#개수 확인
query = """
    SELECT count(*) 
    FROM words_sql
"""

yarn.sql(query).show()

+--------+
|count(1)|
+--------+
|34827421|
+--------+

CPU times: user 64.3 ms, sys: 160 ms, total: 224 ms
Wall time: 1min 12s


#### DataFrame 함수 사용

In [117]:
%%time
#단어별로 분리하고 데이터프레임으로 추출
words_sdf = en_tweets_sdf.rdd.flatMap(text_split).toDF()


CPU times: user 1.47 ms, sys: 23.5 ms, total: 25 ms
Wall time: 1.98 s


In [118]:
%%time
#words_sdf 상위 5개 show
words_sdf.show(5)

+-------------------+----------------+
|               time|            word|
+-------------------+----------------+
|2023-03-14 16:42:09|              RT|
|2023-03-14 16:42:09|        @Mweilc:|
|2023-03-14 16:42:09|           Après|
|2023-03-14 16:42:09|@AmiensMetropole|
|2023-03-14 16:42:09|             fin|
+-------------------+----------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 11.6 ms, total: 11.6 ms
Wall time: 2.07 s


In [121]:
%%time
#개수 확인
#words_sdf.count()


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.05 µs


개수확인할때 Error 발생

### 물리적인 테이블로 보관
- hdfs의 spark-warehour에 저장

In [130]:
!hdfs dfs -rmr spark-warehouse/yarn_twitter_sample_words

rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted spark-warehouse/yarn_twitter_sample_words


In [132]:
!hdfs dfs -expunge

기존폴더 제거

In [133]:
!hdfs dfs -ls spark-warehouse/yarn_twitter_sample_words

ls: `spark-warehouse/yarn_twitter_sample_words': No such file or directory


In [134]:
%%time
#words를 hdfs warehouse에 저장
words_sql.write.saveAsTable('yarn_twitter_sample_words')

CPU times: user 94.4 ms, sys: 92.6 ms, total: 187 ms
Wall time: 1min 26s


`saveAsTable()`함수를 사용하여 `spark-warehouse`에 저장

In [135]:
!hdfs dfs -ls spark-warehouse/yarn_twitter_sample_words | grep _SUCCESS

-rw-r--r--   3 bigdata supergroup          0 2023-03-27 21:26 spark-warehouse/yarn_twitter_sample_words/_SUCCESS


hdfs를 확인해보니 잘 저장되었습니다.

In [136]:
%%time
#hdfs warehouse에 저장된 테이블 개수파악
yarn.table('yarn_twitter_sample_words').count()

CPU times: user 9.9 ms, sys: 7.14 ms, total: 17 ms
Wall time: 11.7 s


34827421

물리적인 테이블의 형태로 보관하니 집계 시간이 빨라짐

#### 단어별로 카운트

기존의 word_sql

In [138]:
%%time
query = '''
    SELECT word, count(*) count
    FROM words_sql
    GROUP BY word
    ORDER BY count DESC
'''

yarn.sql(query).show(5)

+----+-------+
|word|  count|
+----+-------+
|  RT|1232866|
| the| 770781|
|  to| 601721|
|   a| 435703|
| and| 416036|
+----+-------+
only showing top 5 rows

CPU times: user 78.1 ms, sys: 194 ms, total: 272 ms
Wall time: 1min 24s


hdfs에 저장되어있는 물리적인 테이블

In [139]:
%%time
query = '''
    SELECT word, count(*) count
    FROM yarn_twitter_sample_words
    GROUP BY word
    ORDER BY count DESC
'''

yarn.sql(query).show(5)

+----+-------+
|word|  count|
+----+-------+
|  RT|1232866|
| the| 770781|
|  to| 601721|
|   a| 435703|
| and| 416036|
+----+-------+
only showing top 5 rows

CPU times: user 13.6 ms, sys: 18.1 ms, total: 31.8 ms
Wall time: 19.9 s


물리적인 테이블에서 조회할때 훨씬 빠른속도를 보임

#### 데이터 조회

In [140]:
%%time
yarn.sql('SELECT * FROM yarn_twitter_sample_words').show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-17 20:19:19|@krystalwolfy|
|2023-03-17 20:19:19|       Please|
|2023-03-17 20:19:19|         tell|
|2023-03-17 20:19:19|           me|
|2023-03-17 20:19:19|         this|
+-------------------+-------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 9.46 ms, total: 9.46 ms
Wall time: 6.49 s


### 1시간마다 그룹화하여 데이터 집계

In [141]:
%%time
query = '''
 SELECT substr(time, 1, 13) time,
 word,
 count(*) count
 FROM yarn_twitter_sample_words
 GROUP BY time, word
''' 
yarn.sql(query).count()

CPU times: user 8.72 ms, sys: 33.5 ms, total: 42.2 ms
Wall time: 40.8 s


28603658

#### 등록 횟수가 적은 단어 조사

In [142]:
%%time
#등록 횟수가 적은 단어의 수 조사

query = '''
    SELECT t.count, count(*) words
    FROM (
    SELECT word, count(*) count FROM yarn_twitter_sample_words
    GROUP BY word
    ) t
    GROUP BY 1
    ORDER BY 1
'''

yarn.sql(query).show(5)

+-----+-------+
|count|  words|
+-----+-------+
|    1|2259387|
|    2| 321173|
|    3| 134057|
|    4|  75277|
|    5|  48716|
+-----+-------+
only showing top 5 rows

CPU times: user 23.1 ms, sys: 11.3 ms, total: 34.4 ms
Wall time: 21.1 s


#### 단어를 카테고리로 나누는 디멘전 테이블 조회

In [143]:
%%time
# 단어를 카테고리로 나누는 디멘전 테이블

query = '''
 SELECT word, count,
     IF(count > 1000, word, concat('COUNT=', count)) category
 FROM (
     SELECT word, count(*) count FROM yarn_twitter_sample_words
     GROUP BY 1
 ) t
'''

yarn.sql(query).show(10)

+-------------+-----+---------+
|         word|count| category|
+-------------+-----+---------+
|      sane...|    1|  COUNT=1|
|     incoming|  287|COUNT=287|
|          few| 8502|      few|
|         some|28064|     some|
|      danerie|    3|  COUNT=3|
|       waters|  200|COUNT=200|
|       focus…|   18| COUNT=18|
|@Didi70130905|    1|  COUNT=1|
|           By| 3454|       By|
|        trail|  176|COUNT=176|
+-------------+-----+---------+
only showing top 10 rows

CPU times: user 6.83 ms, sys: 47.3 ms, total: 54.1 ms
Wall time: 18.8 s


### 디멘전 테이블을 view: word_category 생성

In [144]:
%%time
yarn.sql(query).createOrReplaceTempView('word_category')

CPU times: user 3.08 ms, sys: 1.94 ms, total: 5.02 ms
Wall time: 40.9 ms


In [145]:
yarn.sql(query).count()

3100631

#### 1시간마다 카테고리별로 그룹화

In [146]:
%%time
# hdfs에 저장되어있는 테이블
# 1시간마다 카테고리별로 그룹화하여 집계
query = '''
 SELECT substr(a.time, 1, 13) time, b.category, count(*) count
 FROM yarn_twitter_sample_words a
 LEFT JOIN word_category b ON a.word = b.word
 GROUP BY 1, 2
'''

yarn.sql(query).show(5)


+-------------+---------+-----+
|         time| category|count|
+-------------+---------+-----+
|2023-03-16 17|COUNT=121|  321|
|2023-03-21 16|  COUNT=3| 8350|
|2023-03-16 00| COUNT=29| 1609|
|2023-03-21 19|COUNT=858|   85|
|2023-03-17 13|COUNT=117|  273|
+-------------+---------+-----+
only showing top 5 rows

CPU times: user 25.8 ms, sys: 60.5 ms, total: 86.2 ms
Wall time: 55.7 s


#### 개수 확인

In [147]:
%%time
yarn.sql(query).count()

CPU times: user 62.3 ms, sys: 42.6 ms, total: 105 ms
Wall time: 58.3 s


220243

### pandas의 DataFrame으로 변환

In [148]:
%%time
result = yarn.sql(query).toPandas()
result.head()

Py4JJavaError: An error occurred while calling o1195.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded


In [49]:
%%time
result.count()

,time,category,count
220238,2023-03-21 00,dare,3
220239,2023-03-16 22,fool,30
220240,2023-03-15 22,haven't,58
220241,2023-03-22 00,picture,6
220242,2023-03-17 12,piece,50
